In [1]:
# Importing the libraries

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

# Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 

import pandasql as psql

In [3]:
# Load the Auto Quote Ins data

AutoIns = pd.read_csv(r"D:\iiit notes\Internship\28 season 21th jun-2021\Auto_Quote_Data_V1.0.csv", header= 0)
AutoIns.head()

,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Marital_Status,Education,Sal_Range,Coverage,Veh_Usage,Annual_Miles_Range,Vehicl_Cost_Range,Re_Quote,Quoted_Premium,Policy_Bind
0,AQ-C-139212,EA,2020/04/25,2020/06/23,2020/05/23,C,2156,Car,3,3,...,Widow,High School,> $ 25 K <= $ 40 K,Balanced,Commute,> 55 K,> $ 10 K <= $ 20 K,No,693.86,Yes
1,AQ-F-136117,EA,2020/02/21,2020/04/20,NaN,F,2153,Van,2,2,...,Dirvorced,Ph.D,> $ 40 K <= $ 60 K,Balanced,Pleasure,> 7.5 K & <= 15 K,<= $ 10 K,No,635.96,No
2,AQ-F-126801,EA,2020/06/19,2020/08/17,2020/07/12,F,2056,Truck,2,1,...,Dirvorced,Ph.D,> $ 40 K <= $ 60 K,Basic,Commute,> 35 K & <= 45 K,> $ 10 K <= $ 20 K,No,780.64,Yes
3,AQ-E-143467,EA,2020/05/02,2020/06/30,2020/05/24,E,2138,Car,1,2,...,Married,Ph.D,> $ 90 K,Basic,Pleasure,<= 7.5 K,<= $ 10 K,No,723.15,Yes
4,AQ-C-143827,EA,2020/02/12,2020/04/11,2020/02/25,C,2327,Truck,3,1,...,Widow,High School,<= $ 25 K,Basic,Pleasure,> 35 K & <= 45 K,<= $ 10 K,No,738.14,Yes


In [4]:
# To Display Dataset information

AutoIns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146259 entries, 0 to 146258
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Quote_Num           146259 non-null  object 
 1   Agent_Type          146259 non-null  object 
 2   Q_Creation_DT       146259 non-null  object 
 3   Q_Valid_DT          146259 non-null  object 
 4   Policy_Bind_DT      32502 non-null   object 
 5   Region              146259 non-null  object 
 6   Agent_Num           146259 non-null  int64  
 7   Policy_Type         146259 non-null  object 
 8   HH_Vehicles         146259 non-null  int64  
 9   HH_Drivers          146259 non-null  int64  
 10  Driver_Age          146259 non-null  int64  
 11  Driving_Exp         146259 non-null  int64  
 12  Prev_Accidents      146259 non-null  int64  
 13  Prev_Citations      146259 non-null  int64  
 14  Gender              146259 non-null  object 
 15  Marital_Status      146259 non-nul

In [5]:
# Convert the target variable data type into interger 

AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].str.replace('Yes', '1') 
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].str.replace('No', '0') 
AutoIns['Policy_Bind'] = AutoIns['Policy_Bind'].astype(int)

In [8]:
# Convert the Date fields into date format 
    
AutoIns['Q_Creation_DT'] = pd.to_datetime(AutoIns['Q_Creation_DT'], format='%Y-%m-%d') 
AutoIns['Q_Valid_DT'] = pd.to_datetime(AutoIns['Q_Valid_DT'], format='%Y-%m-%d')

In [9]:
# Convert the policy bind date into date format 

AutoIns['Policy_Bind_DT'] = pd.to_datetime(AutoIns['Policy_Bind_DT'], format='%Y-%m-%d')

In [10]:
# Adding new columns as Year, Month, Week 
# and Week Day for Q_Creation_DT 

AutoIns['Q_Year'] = AutoIns['Q_Creation_DT'].dt.year 
AutoIns['Q_Month'] = AutoIns['Q_Creation_DT'].dt.month 
AutoIns['Q_WeekNum'] = AutoIns['Q_Creation_DT'].dt.weekofyear 
AutoIns['Q_Day'] = AutoIns['Q_Creation_DT'].dt.weekday

In [12]:
# To Display dataset information after 
AutoIns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146259 entries, 0 to 146258
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Quote_Num           146259 non-null  object        
 1   Agent_Type          146259 non-null  object        
 2   Q_Creation_DT       146259 non-null  datetime64[ns]
 3   Q_Valid_DT          146259 non-null  datetime64[ns]
 4   Policy_Bind_DT      32502 non-null   datetime64[ns]
 5   Region              146259 non-null  object        
 6   Agent_Num           146259 non-null  int64         
 7   Policy_Type         146259 non-null  object        
 8   HH_Vehicles         146259 non-null  int64         
 9   HH_Drivers          146259 non-null  int64         
 10  Driver_Age          146259 non-null  int64         
 11  Driving_Exp         146259 non-null  int64         
 12  Prev_Accidents      146259 non-null  int64         
 13  Prev_Citations      146259 no

In [13]:
# Ranges and new column as 'QP_Range' from 'Quoted_premium' 

AutoIns['QP_Range'] = pd.cut(AutoIns['Quoted_Premium'], [0, 800, 1000, 1200, 9999], 
                             labels=['0-800', '801-1000', '1001-1200', '>1200'])

In [64]:
Data_QP_New01 = psql.sqldf("select Q_Month, Q_Year, count(Q_Month) as Total_Count, sum(Quoted_Premium) as Total_Quoted_Premium \
                            from AutoIns \
                            group by Q_Month, Q_Year \
                            order by Q_Year, Q_Month")
Data_QP_New01.head(15)

,Q_Month,Q_Year,Total_Count,Total_Quoted_Premium
0,10,2019,12255,9123353.86
1,11,2019,11943,8882683.31
2,12,2019,12510,9322840.68
3,1,2020,12342,9171135.52
4,2,2020,11516,8576537.01
5,3,2020,12603,9384782.90
6,4,2020,11882,8831045.57
7,5,2020,12247,9117349.39
8,6,2020,11979,8916547.14
9,7,2020,12368,9184275.55


In [65]:
#Write a file to your local drive 
Data_QP_New01.to_excel('D:/iiit notes/Internship/30 season 24th jun-2021/Quote_Trend_MW_Excel12.xlsx')

In [66]:
Data_QP_New02 = psql.sqldf("select * \
                            from AutoIns \
                            Where Policy_Bind = 1")

Data_QP_New02.head()

,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Annual_Miles_Range,Vehicl_Cost_Range,Re_Quote,Quoted_Premium,Policy_Bind,Q_Year,Q_Month,Q_WeekNum,Q_Day,QP_Range
0,AQ-C-139212,EA,2020-04-25 00:00:00.000000,2020-06-23 00:00:00.000000,2020-05-23 00:00:00.000000,C,2156,Car,3,3,...,> 55 K,> $ 10 K <= $ 20 K,No,693.86,1,2020,4,17,5,0-800
1,AQ-F-126801,EA,2020-06-19 00:00:00.000000,2020-08-17 00:00:00.000000,2020-07-12 00:00:00.000000,F,2056,Truck,2,1,...,> 35 K & <= 45 K,> $ 10 K <= $ 20 K,No,780.64,1,2020,6,25,4,0-800
2,AQ-E-143467,EA,2020-05-02 00:00:00.000000,2020-06-30 00:00:00.000000,2020-05-24 00:00:00.000000,E,2138,Car,1,2,...,<= 7.5 K,<= $ 10 K,No,723.15,1,2020,5,18,5,0-800
3,AQ-C-143827,EA,2020-02-12 00:00:00.000000,2020-04-11 00:00:00.000000,2020-02-25 00:00:00.000000,C,2327,Truck,3,1,...,> 35 K & <= 45 K,<= $ 10 K,No,738.14,1,2020,2,7,2,0-800
4,AQ-F-130892,EA,2019-12-12 00:00:00.000000,2020-02-09 00:00:00.000000,2020-01-09 00:00:00.000000,F,2309,Truck,1,2,...,> 25 K & <= 35 K,> $ 10 K <= $ 20 K,No,707.06,1,2019,12,50,3,0-800


In [68]:
AutoIns['NP_Date'] = pd.to_datetime(AutoIns['Policy_Bind_DT'])
AutoIns['NP_Month'] = AutoIns['Policy_Bind_DT'].dt.month

In [69]:
# Adding new columns as Year, Month, Week # and Week Day for Q_Creation_DT
AutoIns['NP1_Year'] = AutoIns['Policy_Bind_DT'].dt.year
AutoIns['NP1_Month'] = AutoIns['Policy_Bind_DT'].dt.month
AutoIns['NP1_WeekNum'] = AutoIns['Policy_Bind_DT'].dt.weekofyear
AutoIns['NP1_Day'] = AutoIns['Policy_Bind_DT'].dt.weekday

In [70]:
AutoIns.head()


,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Q_Month,Q_WeekNum,Q_Day,QP_Range,NP_Date,NP_Month,NP1_Year,NP1_Month,NP1_WeekNum,NP1_Day
0,AQ-C-139212,EA,2020-04-25,2020-06-23,2020-05-23,C,2156,Car,3,3,...,4,17,5,0-800,2020-05-23,5.0,2020.0,5.0,21.0,5.0
1,AQ-F-136117,EA,2020-02-21,2020-04-20,NaT,F,2153,Van,2,2,...,2,8,4,0-800,NaT,NaN,NaN,NaN,NaN,NaN
2,AQ-F-126801,EA,2020-06-19,2020-08-17,2020-07-12,F,2056,Truck,2,1,...,6,25,4,0-800,2020-07-12,7.0,2020.0,7.0,28.0,6.0
3,AQ-E-143467,EA,2020-05-02,2020-06-30,2020-05-24,E,2138,Car,1,2,...,5,18,5,0-800,2020-05-24,5.0,2020.0,5.0,21.0,6.0
4,AQ-C-143827,EA,2020-02-12,2020-04-11,2020-02-25,C,2327,Truck,3,1,...,2,7,2,0-800,2020-02-25,2.0,2020.0,2.0,9.0,1.0


In [71]:
Data_QP_New03 = psql.sqldf("select * \
 from AutoIns \
 Where Policy_Bind = 1")

Data_QP_New03.head()

,Quote_Num,Agent_Type,Q_Creation_DT,Q_Valid_DT,Policy_Bind_DT,Region,Agent_Num,Policy_Type,HH_Vehicles,HH_Drivers,...,Q_Month,Q_WeekNum,Q_Day,QP_Range,NP_Date,NP_Month,NP1_Year,NP1_Month,NP1_WeekNum,NP1_Day
0,AQ-C-139212,EA,2020-04-25 00:00:00.000000,2020-06-23 00:00:00.000000,2020-05-23 00:00:00.000000,C,2156,Car,3,3,...,4,17,5,0-800,2020-05-23 00:00:00.000000,5.0,2020.0,5.0,21.0,5.0
1,AQ-F-126801,EA,2020-06-19 00:00:00.000000,2020-08-17 00:00:00.000000,2020-07-12 00:00:00.000000,F,2056,Truck,2,1,...,6,25,4,0-800,2020-07-12 00:00:00.000000,7.0,2020.0,7.0,28.0,6.0
2,AQ-E-143467,EA,2020-05-02 00:00:00.000000,2020-06-30 00:00:00.000000,2020-05-24 00:00:00.000000,E,2138,Car,1,2,...,5,18,5,0-800,2020-05-24 00:00:00.000000,5.0,2020.0,5.0,21.0,6.0
3,AQ-C-143827,EA,2020-02-12 00:00:00.000000,2020-04-11 00:00:00.000000,2020-02-25 00:00:00.000000,C,2327,Truck,3,1,...,2,7,2,0-800,2020-02-25 00:00:00.000000,2.0,2020.0,2.0,9.0,1.0
4,AQ-F-130892,EA,2019-12-12 00:00:00.000000,2020-02-09 00:00:00.000000,2020-01-09 00:00:00.000000,F,2309,Truck,1,2,...,12,50,3,0-800,2020-01-09 00:00:00.000000,1.0,2020.0,1.0,2.0,3.0


In [72]:
Data_QP_New03.shape

(32502, 36)

In [73]:
Data_QP_New04 = psql.sqldf("select NP1_Month, NP1_Year, count(NP1_Month) as Total_Count, \
 sum(Quoted_Premium) as Total_Policy_Premium \
 from Data_QP_New03 \
 Where Policy_Bind = 1 \
 group by NP1_Month, NP1_Year \
 order by NP1_Year, NP1_Month")

Data_QP_New04.head(15)

,NP1_Month,NP1_Year,Total_Count,Total_Policy_Premium
0,10.0,2019.0,1246,930217.43
1,11.0,2019.0,2635,1960671.03
2,12.0,2019.0,2690,1997855.71
3,1.0,2020.0,2735,2037486.82
4,2.0,2020.0,2559,1898735.22
5,3.0,2020.0,2815,2087555.90
6,4.0,2020.0,2686,2000858.32
7,5.0,2020.0,2792,2078497.18
8,6.0,2020.0,2576,1914441.07
9,7.0,2020.0,2822,2100074.52


In [74]:
Data_QP_New04.to_excel('D:/iiit notes/Internship/30 season 24th jun-2021/NP_Policy_New99.xlsx')